# Assessing the Skill of the Climatological Ensemble with the Brier Score (BS)
Version 19 January 2024, Selina Kiefer

### Input: csv-files
ensemble of continuous timeseries of ground truth temperature for a winter (e.g. climatological ensemble) in csv-format, binary timeseries of cold wave days in csv-format
### Output: csv-file, png-files
binary timeseries of cold wave days in the climatological ensemble, continuous timeseries of daily BS values in csv-format in csv-format and plotted in png-format as well as the prediction of the climatological ensemble plotted together with the ground truth in png-format

#### Set the paths' to the defined functions and configuration file and set its name

In [ ]:
# Set the path to the defined functions.
PATH_defined_functions = './Defined_Functions/'

In [ ]:
# Set the path and name of the configuration file.
PATH_configurations = './Configurations/'
ifile_configurations = 'Configurations_Skill_Assessment_Climatological_Ensemble_with_BS.yaml'

#### Import the necessary python packages and functions

In [ ]:
# Import the necessary python packages.
import yaml
import numpy as np
import calendar
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
# Import the necessary defined functions.
import sys
sys.path.insert(1, PATH_defined_functions)
from read_in_csv_data import *
from truncate_data_by_date import*
from create_auxiliary_date import *
from apply_cold_wave_definition_smid_et_al_2019 import *

#### Read in the configuration file and the data specified in it

In [ ]:
# Read in the configuration file.
with open(PATH_configurations+ifile_configurations) as f:
    config = yaml.safe_load(f)

In [ ]:
# Read in the climatological_ensemble and remove any unnamed columns as well as the index column.
df_climatological_ensemble = read_in_csv_data(config['PATH_climatological_ensemble'], config['ifile_climatological_ensemble'])
df_climatological_ensemble = df_climatological_ensemble.loc[:, ~df_climatological_ensemble.columns.str.contains('^Unnamed')]
df_climatological_ensemble = df_climatological_ensemble.drop(['index'], axis =1 )

In [ ]:
# Set the name of the columns containing the time and the variables of the climatological_ensemble.
time_column_name_climatological_ensemble = df_climatological_ensemble.columns[0]
var_column_name_climatological_ensemble = df_climatological_ensemble.columns[1:]

In [ ]:
# Check that everything is selected correctly.
print('Names of climatological_ensemble done by the ML model: ')
print(var_column_name_climatological_ensemble)
print('Name of the column containing the time: ')
print(time_column_name_climatological_ensemble)
print('Dataframe containing the climatological ensemble: ')
df_climatological_ensemble.head()

In [ ]:
# Read in the ground truth and remove any unnamed columns as well as the index column.
df_ground_truth = read_in_csv_data(config['PATH_ground_truth'], config['ifile_ground_truth'])
df_ground_truth = df_ground_truth.loc[:, ~df_ground_truth.columns.str.contains('^Unnamed')]
df_ground_truth = df_ground_truth.drop(['index'], axis =1 )

In [ ]:
# Set the name of the columns containing the time and the variables of the ground truth.
time_column_name_ground_truth = df_ground_truth.columns[0]
var_column_name_ground_truth = df_ground_truth.columns[1]

In [ ]:
# Check that everything is selected correctly.
print('Ground truth to compare the climatological ensemble with: ')
print(var_column_name_ground_truth)
print('Name of the column containing the time: ')
print(time_column_name_ground_truth)
print('Dataframe containing the ground truth: ')
df_ground_truth.head()

In [ ]:
# Read in the cold wave thresholds and remove any unnamed columns as well as the index column.
df_thresholds = read_in_csv_data(config['PATH_thresholds'], config['ifile_thresholds'])
df_thresholds = df_thresholds.loc[:, ~df_thresholds.columns.str.contains('^Unnamed')]
df_thresholds = df_thresholds.drop(['index'], axis =1 )

In [ ]:
# Set the name of the columns containing the time and the threshold.
time_column_name_thresholds = df_thresholds.columns[0]
var_column_name_thresholds = df_thresholds.columns[1]

In [ ]:
# Check that everything is selected correctly.
print('Thresholds for cold waves to apply to the climatological ensemble: ')
print(var_column_name_thresholds)
print('Name of the column containing the time: ')
print(time_column_name_thresholds)
print('Dataframe containing the thresholds: ')
df_thresholds.head()

#### Prepare the thresholds for the cold waves, the climatological ensemble and the ground truth for the skill assessment 

In [ ]:
# A list with all the start years of the winters in the validation period is created. 
start_years_of_winter = np.arange(config['start_year_of_first_winter'], config['start_year_of_last_winter']+1)

In [ ]:
# At first, two different dataframes are created with the threshold for the cold wave 
# definition. One for regular years and one for leap years. Therefore, the index of the original
# dataframe is set to the time and the index of the 29 February is determined. Then, a new 
# dataframe without the 29 February is created for regular years. The original dataframe is used
# for leap years.
df_thresholds[time_column_name_thresholds]=pd.to_datetime(df_thresholds[time_column_name_thresholds])
df_thresholds = df_thresholds.set_index(time_column_name_thresholds)
index_of_february_29 = df_thresholds[((df_thresholds.index.month == 2) & (df_thresholds.index.day == 29))].index
df_thresholds_without_29_feb = df_thresholds.drop(index_of_february_29)
df_thresholds = df_thresholds.reset_index()
df_thresholds_without_29_feb = df_thresholds_without_29_feb.reset_index()

In [ ]:
# In a next step, two different dataframes are created with climatological ensemble. One for 
# regular years and one for leap years. Therefore, the index of the original dataframe is set 
# to the time and the index of the 29 February is determined. Then, a new  dataframe without 
# the 29 February is created for regular years. The original dataframe is used for leap years.
df_climatological_ensemble[time_column_name_climatological_ensemble]=pd.to_datetime(df_climatological_ensemble[time_column_name_climatological_ensemble])
df_climatological_ensemble = df_climatological_ensemble.set_index(time_column_name_climatological_ensemble)
index_of_february_29 = df_climatological_ensemble[((df_climatological_ensemble.index.month == 2) & (df_climatological_ensemble.index.day == 29))].index
df_climatological_ensemble_without_29_feb = df_climatological_ensemble.drop(index_of_february_29)
df_climatological_ensemble = df_climatological_ensemble.reset_index()
df_climatological_ensemble_without_29_feb = df_climatological_ensemble_without_29_feb.reset_index()

In [ ]:
# Then, the needed thresholds for the respective winter are taken and the respective
# winter is extracted from the ground truth and the climatological_ensemble. Then, the cold wave criterion
# by Smid et al. (2019) is applied to the climatological_ensemble. In the end, lists
# containing the ground truth, the fraction of the climatological ensemble showing a cold 
# wave and the forecast dates are created.
forecast_dates = []

list_members_cold_waves_climatological_ensemble = []
list_cold_waves_climatological_ensemble = []

all_winters_list_cold_waves_ground_truth = []
all_winters_list_cold_waves_climatological_ensemble = []

for start_year_of_winter in start_years_of_winter:
    
    if calendar.isleap(start_year_of_winter+1):
        threshold_cold_waves = df_thresholds[var_column_name_thresholds]
    else:
        threshold_cold_waves = df_thresholds_without_29_feb[var_column_name_thresholds]
    
    start_winter = datetime(start_year_of_winter, config['start_month_winter'], config['start_day_winter'])
    end_winter = datetime(start_year_of_winter+1, config['end_month_winter'], config['end_day_winter'])

    df_ground_truth_respective_winter = truncate_data_by_date(df_ground_truth, time_column_name_ground_truth, start_winter.strftime('%Y_%m_%d'), end_winter.strftime('%Y_%m_%d')) 
    
    list_cold_waves_ground_truth = df_ground_truth_respective_winter[var_column_name_ground_truth]
    
    if calendar.isleap(start_year_of_winter+1):
        df_climatological_ensemble_respective_winter = df_climatological_ensemble   
    else:
        df_climatological_ensemble_respective_winter = df_climatological_ensemble_without_29_feb
    
    df_climatological_ensemble_time_column = pd.DataFrame()    
    df_climatological_ensemble_time_column['time'] = df_climatological_ensemble_respective_winter[time_column_name_climatological_ensemble]
       
    list_members_cold_waves_climatological_ensemble = []
    list_cold_waves_climatological_ensemble = []
    
    for i in var_column_name_climatological_ensemble:
        members_with_cold_waves = apply_cold_wave_definition_smid_et_al_2019(df_climatological_ensemble_time_column, df_climatological_ensemble_respective_winter, i, threshold_cold_waves)          
        list_members_cold_waves_climatological_ensemble.append(members_with_cold_waves)
        
    fraction_of_member_with_cold_waves = np.sum(list_members_cold_waves_climatological_ensemble, axis=0)
    list_cold_waves_climatological_ensemble.append(fraction_of_member_with_cold_waves/len(var_column_name_climatological_ensemble))
    
    all_winters_list_cold_waves_ground_truth.append(list_cold_waves_ground_truth)
    all_winters_list_cold_waves_climatological_ensemble.append(np.squeeze(list_cold_waves_climatological_ensemble))
    
    forecast_dates.append(pd.to_datetime(df_ground_truth_respective_winter[time_column_name_ground_truth]))

#### Prepare the cold wave prediction of the climatological for saving in csv-format

In [ ]:
# Bring the 29th February NaN entry for regular years to its right position (right now, it is
# appended at the end of the timeseries, since the timeseries for regular years is one day short,
# but this will correspond to 31 December later)
cold_waves_with_nan_at_position_of_29_february = []
for k in range(len(start_years_of_winter)):
    if calendar.isleap(start_years_of_winter[k]+1)==False:
        cold_waves_leap_year = list(all_winters_list_cold_waves_climatological_ensemble[k])
        cold_waves_leap_year.insert(120, np.nan)
        cold_waves_leap_year = cold_waves_leap_year[0:len(cold_waves_leap_year)]
        cold_waves_with_nan_at_position_of_29_february.append(cold_waves_leap_year)
    else:
        cold_waves_with_nan_at_position_of_29_february.append(all_winters_list_cold_waves_climatological_ensemble[k])

In [ ]:
# For an easier handling of the data later on, a "auxiliary date" is created. This is simply a
# timeseries of dates of a leap year winter (here 2003/2004), which is afterwards sorted 
# chronologically by month (Jan-Dec). The exact year itself does not matter since only the month
# and day are relevant for the climatological ensemble and furthermore only these two will be 
# shown on a plot.
auxiliary_time = create_auxiliary_date(config['start_month_winter'], config['start_day_winter'], config['end_month_winter'], config['end_day_winter'])

In [ ]:
# The list with the predicted cold wave days of the climatological ensemble is converted into a
# pandas dataframe and then transposed for a better representation of the data. In a next step,
# the time information is added again to the dataframe. Since the climatological_ensemble are the same for
# every winter, only 1 column is needed. 
df_climatological_ensemble_cold_waves = pd.DataFrame(cold_waves_with_nan_at_position_of_29_february)
df_climatological_ensemble_cold_waves = df_climatological_ensemble_cold_waves.T
df_climatological_ensemble_cold_waves['auxiliary_time'] = pd.to_datetime(auxiliary_time)
df_climatological_ensemble_cold_waves = df_climatological_ensemble_cold_waves[['auxiliary_time', 0]]
df_climatological_ensemble_cold_waves = df_climatological_ensemble_cold_waves.rename(columns={0:'Fraction of Ensembles Members \n Predicting Cold Wave Day'})

#### Save the climatological ensemble's cold wave days prediction in csv-format

In [ ]:
# Save the days of the cold wave climatological_ensemble of the climatological ensemble in csv-format.
df_climatological_ensemble_cold_waves.to_csv(config['PATH_output_files']+'daily_climatological_ensemble_cold_waves_'+var_column_name_ground_truth+'_'+str(config['start_year_of_first_winter'])+'_'+str(config['start_year_of_last_winter']+1)+'.csv') 

#### Calculation of the BS between the ground truth and the climatological ensemble

In [ ]:
# Now, the daily BS is computed and saved to a list. Furthermore, the forecast time is saved in
# continuous form to a list.
forecast_time = []
bs_daily_one_year = []
bs = []
bs_winterwise = []

for k in range(len(start_years_of_winter)):
    forecast_time.extend(forecast_dates[k])
    bs_one_year = 0
    bs_daily_one_year = []
    for l in range(len(all_winters_list_cold_waves_climatological_ensemble[k])):
        bs_daily = (all_winters_list_cold_waves_climatological_ensemble[k][l]-all_winters_list_cold_waves_ground_truth[k][l])**2
        bs_daily_one_year.append(bs_daily)
    bs.extend(np.array(bs_daily_one_year))
    bs_winterwise.append(np.array(bs_daily_one_year))

In [ ]:
# Then, a dataframe containing the daily BS values and the corresponding forecasting times is
# created. 
df_skill_measure_bs = pd.DataFrame()
df_skill_measure_bs['time'] = forecast_time
df_skill_measure_bs['BS'] = np.array(bs)

#### Save the BS in csv-format

In [ ]:
# This pandas dataframe containing the BS values is saved in csv format. 
df_skill_measure_bs.to_csv(config['PATH_statistics']+config['model_name']+'_BS_ground_truth_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_'+str(config['start_year_of_first_winter'])+'_'+str(config['start_year_of_last_winter']+1)+'.csv')

#### Visualizing the predictions of the climatological ensemble together with the ground truth and the BS for a plausibility check

In [ ]:
# Before plotting, the information about the input data which should be shown in the plot title
# is converted to a nice-looking string by creating the line-breaks set in the configuration 
# file.
str_input_info_for_plot_titles = config['input_data_title']
str_input_info_for_plot_titles = str_input_info_for_plot_titles.replace('|', '\n')

In [ ]:
# For illustration purposes, the fraction of ensemble members of the climatological ensemble predicting a cold wave
# day is plotted with the cold waves in the ground truth. This gives a first impression about
# the model's forecast skill.
for i in range(len(start_years_of_winter)):
    fig = plt.subplots()
    plt.plot(forecast_dates[i], all_winters_list_cold_waves_ground_truth[i], color='k', linestyle='--', label='Ground Truth')
    plt.plot(forecast_dates[i], all_winters_list_cold_waves_climatological_ensemble[i], marker='o', linestyle='', color='r', alpha=0.6, label='climatological_ensemble')    
    plt.legend(bbox_to_anchor=(0, -0.15), loc='upper left')
    plt.xlabel(time_column_name_ground_truth)
    plt.ylabel('Fraction of Ensemble Members \n Predicting a Cold Wave')
    plt.title(config['model_name']+' climatological_ensemble:\n '+str_input_info_for_plot_titles)
    plt.savefig(config['PATH_plots']+config['model_name']+'_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_'+str(start_years_of_winter[i])+'_'+str(start_years_of_winter[i]+1)+'.png', bbox_inches='tight')

In [ ]:
# The BS values for each winter are plotted separately. In combination with the plot above a first plausibility
# check is possible. The lower the BS value, the more similar the prediction of the climatological 
# ensemble and the ground truth have to be.
for n in range(len(start_years_of_winter)):
    fig = plt.subplots()
    plt.plot(forecast_dates[n], bs_winterwise[n], color='r', marker='o', markersize=4, linestyle='--')
    plt.axhline(y=np.nanmean(bs_winterwise[n]), color='grey', linestyle='-', label='Wintermean')
    plt.legend(bbox_to_anchor=(0, -0.15), loc='upper left')
    plt.xlabel(time_column_name_ground_truth)
    plt.ylabel('BS')
    plt.title('Daily BS of '+config['model_name']+' climatological_ensemble:\n '+str_input_info_for_plot_titles)
    plt.savefig(config['PATH_plots']+config['model_name']+'_BS_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_'+str(start_years_of_winter[n])+'_'+str(start_years_of_winter[n]+1)+'.png', bbox_inches='tight')

#### Visualizing the BS for all winters in the evaluation period for a quick overview of the forecasting performance of the climatological ensemble

In [ ]:
# The timeseries of the daily BS values is plotted for all winters in the evaluation period. 
plt.plot(forecast_time, bs, marker='s', linestyle='', markersize=2, color='r')
plt.xlabel(time_column_name_ground_truth)
plt.ylabel('BS')
plt.title('Daily BS of '+config['model_name']+' climatological_ensemble: \n '+str_input_info_for_plot_titles)
plt.savefig(config['PATH_plots']+config['model_name']+'_BS_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_'+str(config['start_year_of_first_winter'])+'_'+str(config['start_year_of_last_winter']+1)+'.png', bbox_inches='tight')

In [ ]:
# End of Program